In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Activation
import sklearn.neighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import svm
from sklearn.svm import SVR
import sklearn.model_selection
import numpy.random as rd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import warnings, pandas as pd, numpy as np, time, math, configparser, random


In [2]:
data = pd.read_csv ('/Breast Cancer Data.csv')
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [4]:
data['diagnosis'].loc[(data['diagnosis'] == 'M')] = 1 #злоякісний
data['diagnosis'].loc[(data['diagnosis'] == 'B')] = 0

In [5]:
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

In [6]:
X = data
Y = X.pop('diagnosis')
data = X

In [7]:
#
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)

In [ ]:
import joblib
scaler_filename = "scaler.save" #save MinMaxScaler
joblib.dump(scaler, scaler_filename) 

In [8]:
X = data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((398, 30), (398,), (171, 30), (171,))

In [10]:
print("in Train ",'доброякісних:',len(np.where(y_train == 0)[0]),', злоякісних:', len(np.where(y_train == 1)[0]))
print("in Test ",'доброякісних:',len(np.where(y_test == 0)[0]),', злоякісних:', len(np.where(y_test == 1)[0]))

in Train  доброякісних: 249 , злоякісних: 149
in Test  доброякісних: 108 , злоякісних: 63


In [11]:
X_train = np.array(X_train)
y_train = np.array(y_train).astype('int')
X_test = np.array(X_test)
y_test= np.array(y_test).astype('int')

In [12]:
def sanitized_gwo(X_train, X_test, y_train, y_test, SearchAgents_no, T, dim, lb, ub):
    Alpha_position = [0, 0]  # Initialize the position of Alpha Wolf
    Beta_position = [0, 0]
    Delta_position = [0, 0]

    Alpha_score = float("inf")  # Initialize the value of Alpha Wolf's objective function
    Beta_score = float("inf")
    Delta_score = float("inf")

    Positions = np.dot(rd.rand(SearchAgents_no, dim), (ub - lb)) + lb  # initialize the first search position

    Convergence_curve = np.zeros((1, T))  # initialization fusion curve

    iterations = []
    accuracy = []

    # Main Loop
    t = 0
    while t < T:

        # Iterate over each wolf
        for i in range(0, (Positions.shape[0])):
            # If the search position exceeds the search space, you need to return to the search space
            for j in range(0, (Positions.shape[1])):
                Flag4ub = Positions[i, j] > ub
                Flag4lb = Positions[i, j] < lb
                # If the wolf's position is between the maximum and minimum, the position does not need to be adjusted, if it exceeds the maximum, the maximum returns to the maximum value boundary

                if Flag4ub:
                    Positions[i, j] = ub
                if Flag4lb:
                    Positions[i, j] = lb
            '''SVM MODEL TRAINING - FOR CLASSIFICATION PROBLEM DATASET'''
            rbf_svm = svm.SVC(kernel = 'rbf', C = Positions[i][0], gamma = Positions[i][1]).fit(X_train, y_train)  #svm
            cv_accuracies = cross_val_score(rbf_svm,X_test,y_test,cv =3,scoring = 'accuracy')


            # To minimize the error rate
            accuracies = cv_accuracies.mean()
            fitness_value = (1 - accuracies) * 100
            if fitness_value < Alpha_score:  # If the objective function value is less than the objective function value of Alpha Wolf
                Alpha_score = fitness_value  # Then update the target function value of Alpha Wolf to the optimal target function value
                Alpha_position = Positions[
                    i]  # At the same time update the position of the Alpha wolf to the optimal position
            if fitness_value > Alpha_score and fitness_value < Beta_score:  # If the objective function value is between the objective function value of Alpha Wolf and Beta Wolf
                Beta_score = fitness_value  # Then update the target function value of Beta Wolf to the optimal target function value
                Beta_position = Positions[i]
            if fitness_value > Alpha_score and fitness_value > Beta_score and fitness_value < Delta_score:  # If the target function value is between the target function value of Beta Wolf and Delta Wolf
                Delta_score = fitness_value  # Then update the target function value of Delta Wolf to the optimal target function value
                Delta_position = Positions[i]

        a = 2 - t * (2 / T)

        # Iterate over each wolf
        for i in range(0, (Positions.shape[0])):
            # Traverse through each dimension
            for j in range(0, (Positions.shape[1])):
                # Surround prey, location update
                r1 = rd.random(1)  # Generate a random number between 0 ~ 1
                r2 = rd.random(1)
                A1 = 2 * a * r1 - a  # calculation factor A
                # C1=2*r2 # calculation factor C
                C1 = 0.5 + (0.5 * math.exp(-j / 500)) + (
                            1.4 * (math.sin(j) / 30))  # Time varying Acceleration constant

                # Alphawolf location update

                D_alpha = abs(C1 * Alpha_position[j] - Positions[i, j])
                X1 = Alpha_position[j] - A1 * D_alpha

                r1 = rd.random(1)
                r2 = rd.random(1)

                A2 = 2 * a * r1 - a
                # C2=2*r2
                C2 = 1 + (1.4 * (1 - math.exp(-j / 500))) + (
                            1.4 * (math.sin(j) / 30))  # Difference Mean based Perturbation time varying parameter

                # Beta wolf location update
                D_beta = abs(C2 * Beta_position[j] - Positions[i, j])
                X2 = Beta_position[j] - A2 * D_beta
                r1 = rd.random(1)
                r2 = rd.random(1)

                A3 = 2 * a * r1 - a
                # C3=2*r2
                C3 = (1 / (1 + math.exp(-0.0001 * j / T))) + (
                            (0.5 - 2.5) * ((j / T) ** 2))  # sigmoid-based acceleration coefficient

                # Delta Wolf Location Update
                D_delta = abs(C3 * Delta_position[j] - Positions[i, j])
                X3 = Delta_position[j] - A3 * D_delta

                # Location update
                Positions[i, j] = (X1 + X2 + X3) / 3

        t = t + 1
        iterations.append(t)
        accuracy.append((100 - Alpha_score) / 100)
        print('----------------Count of iterations----------------' + str(t))
        print(Positions)
        print('C and gamma:' + str(Alpha_position))
        print('accuracy:' + str((100 - Alpha_score) / 100))

    best_C = Alpha_position[0]
    best_gamma = Alpha_position[1]

    return best_C, best_gamma, iterations, accuracy


In [13]:
SearchAgents_no = 20  # Number of Wolfs
T = 20  # maximum number of iterations
dim = 2  # Need to optimize two variables - Cost and Gamma
lb = 0.01  # lower bound Parameter
ub = 10  # upper bound Parameter
best_C, best_gamma, iterations, accuracy = sanitized_gwo(X_train, X_test, y_train, y_test, SearchAgents_no, T, dim, lb, ub)

----------------Count of iterations----------------1
[[ 5.08170032  1.94724158]
 [-0.83207044  3.24516906]
 [ 2.16054518  1.16939269]
 [ 4.36464087  2.31403188]
 [ 5.90773821  1.47508791]
 [ 2.82865327  2.05441279]
 [ 4.35094766  1.31445292]
 [ 8.49040249  2.86879915]
 [ 5.80287973  1.23536242]
 [ 6.23641514  4.32509759]
 [ 2.99930177  0.53975906]
 [ 4.51282309  0.45485972]
 [ 5.65318987  0.74010461]
 [ 4.4911057   1.78438182]
 [ 1.50469529  3.65998687]
 [ 4.99462626  0.03116993]
 [ 3.70439296  3.57952992]
 [ 1.88962772  0.95879326]
 [ 5.08659386  1.7198482 ]
 [ 4.41141275  3.49990441]]
C and gamma:[1.88962772 0.95879326]
accuracy:0.9766081871345028
----------------Count of iterations----------------2
[[ 3.47247035  1.72017272]
 [ 1.24741216  1.35686525]
 [ 1.88476274  1.47952964]
 [ 1.35659603  2.74842116]
 [ 0.04373176  1.69079435]
 [ 2.18416309  2.00858254]
 [ 3.06180436  1.39401198]
 [-3.44369546  2.61678007]
 [ 0.57490272  2.1166743 ]
 [ 2.58590386  2.38321416]
 [ 2.72674586  1.60

----------------Count of iterations----------------15
[[0.91209699 3.75698619]
 [0.86302799 4.16599391]
 [0.89059997 3.7323048 ]
 [0.91514836 3.9148957 ]
 [0.86223251 3.93988117]
 [0.88539762 4.22507704]
 [0.94120521 3.68497001]
 [0.85790459 4.20309881]
 [0.8493827  3.91997105]
 [0.85135654 3.79431073]
 [0.85272532 3.7392562 ]
 [0.89737407 3.79059864]
 [0.90914899 3.89011041]
 [0.8237224  4.00670128]
 [0.93278359 4.21867504]
 [0.8655604  3.92764877]
 [0.92749279 3.83680889]
 [0.86279985 4.21220131]
 [0.87142119 3.85644088]
 [0.86051197 3.83564341]]
C and gamma:[0.86279985 4.21220131]
accuracy:0.9766081871345028
----------------Count of iterations----------------16
[[0.81307966 4.26617817]
 [0.92171684 3.95262224]
 [0.90449997 4.57639178]
 [0.91090433 4.18647082]
 [0.89234498 4.12095449]
 [0.82129567 4.02543614]
 [0.75366916 4.13609084]
 [0.76552781 4.19848432]
 [0.84700593 4.08589027]
 [0.8475065  4.12287596]
 [0.76094828 3.95759172]
 [0.90089865 3.88781776]
 [0.85985449 4.15423199]
 [

In [14]:
model = SVC(kernel='linear', C=best_C, gamma=best_gamma, probability=True)
model.fit(X_train, y_train)

SVC(C=0.8210495081686259, gamma=4.287572517256791, kernel='linear',
    probability=True)

In [17]:
#import sklearn.metrics as metrics
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=sklearn.metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=sklearn.metrics.mean_absolute_error(y_true, y_pred)
    mse=sklearn.metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=sklearn.metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=sklearn.metrics.median_absolute_error(y_true, y_pred)
    r2=sklearn.metrics.r2_score(y_true, y_pred)


    print('r2(коефіцієнт детермінації): ', round(r2,4))
    print('MAE(середня абсолютна похибка): ', round(mean_absolute_error,4))
    print('MSE(середньоквадратична похибка): ', round(mse,4))
    print('RMSE(середньоквадратичне відхилення): ', round(np.sqrt(mse),4))

In [18]:
regression_results(y_test, model.predict(X_test))

r2(коефіцієнт детермінації):  0.9497
MAE(середня абсолютна похибка):  0.0117
MSE(середньоквадратична похибка):  0.0117
RMSE(середньоквадратичне відхилення):  0.1081


In [19]:
accuracy_score(y_test,model.predict(X_test))

0.9883040935672515

In [ ]:
#save model
import pickle
filename = '/model/model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# load the model from disk
model = pickle.load(open(filename, 'rb'))


In [ ]:
#load scaler
scaler_2 = joblib.load("scaler.save") 